# Tutorial 7: Introducción a Large Language Models (LLM).

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Felipe Villena](https://fabianvillena.cl/).
- Profesor Auxiliar: [Gabriel Iturra](https://giturra.cl/)


### Objetivos del Tutorial

- Entender que son los LLM.
- Comprender como utilizar la API de OpenAI.
- Que es Zero Shot Learning, y como utilizarlo.
- Que es Few Shot Learning, y como utilizarlo..
- Conocer la librería `scikit-learn`.

In [1]:
!pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/69/95/22a9a81cebd54e18841da429f05f06ed867648768f7af938ad34f13197fd/openai-1.3.3-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/82/61/a5fca4a1e88e40969bbd0cf0d981f3aa76d5057db160b94f49603fc18740/httpx-0.25.1-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore from https://files.pythonhosted.org/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9840b6202a93d9a0f/httpcore-1.0.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------------------------------- 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/220.3 kB ? eta -:--:--
   --------------------------------------- 220.3/220.3 kB 13.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.0 kB ? eta -:--:--
   -------------------

## Importar las librerías.

In [2]:
import pandas as pd
from openai import OpenAI

## Cargar el dataset

In [ ]:
!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/tutoriales/data/ag_news/train.csv

--2023-11-18 20:27:16--  https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/tutoriales/data/ag_news/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28946992 (28M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  27.61M  --.-KB/s    in 0.1s    

2023-11-18 20:27:17 (188 MB/s) - ‘train.csv’ saved [28946992/28946992]



In [ ]:
train_df = pd.read_csv('train.csv')

In [ ]:
train_df.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [ ]:
labels = ['World','Sports', 'Business', 'Sci/Tech']

In [ ]:
def map_label(class_index):
    return labels[class_index - 1]

In [ ]:
train_df['Class Name'] = train_df['Class Index'].apply(map_label)

In [ ]:
train_df.sample(5)

,Class Index,Title,Description,Class Name
72877,4,'Grand Theft Auto' Seen Breaking Games Sales R...,&lt;p&gt;\&lt;/p&gt;&lt;p&gt; By Franklin Paul...,Sci/Tech
28567,4,"Mayhem, for fun",I admit to being skeptical that the Teen-rated...,Sci/Tech
40984,3,Counting the cost of Hurricane Ivan,Hurricane Ivan has left a trail of destroyed h...,Business
78644,3,Australia #39;s Incitec Pivot FY04 Net A\$75.1...,MELBOURNE (Dow Jones)--Australian fertilizer i...,Business
63259,3,Dollar Slips Toward Lows After Weak Data,TOKYO (Reuters) - The dollar edged down again...,Business


## Usabilidad de la API de OpenAI para interactuar con Llama

In [3]:
from openai import OpenAI

client = OpenAI(
    api_key="",
    base_url="https://api.deepinfra.com/v1/openai"
)



### Zero Shot Learning

In [ ]:
content1 = f"I will provide you with a dataset of news articles, which you must classify correctly. The labels are {labels[0]}, {labels[1]}, {labels[2]}, and {labels[3]}, and please list the article and its the label in your answer."
content1

'I will provide you with a dataset of news articles, which you must classify correctly. The labels are World, Sports, Business, and Sci/Tech, and please list the article and its the label in your answer.'

In [ ]:
new_examples = train_df.sample(5)

In [ ]:
descriptions = list(new_examples['Description'])

content2 = ''

for idx, new in enumerate(descriptions):
  content2 += f'New {idx + 1}: {new}\n'


In [ ]:
print(content2)

New 1:  TOKYO (Reuters) - The Nikkei average was flat in  mid-morning trade on Tuesday, with a rise in bank shares and  insurers helping the Tokyo market stabilize after the  benchmark's slide to a five-month closing low the previous day.
New 2: Analysts say the move could legally bind the Communist Party leadership to follow through on its long-standing threat to attack Taiwan if the island formally declares independence.
New 3: Libyan President Moammar Gadhafi has appealed for the release of British engineer Kenneth Bigley, who was taken hostage in Iraq three weeks ago.
New 4: Yukos can succeed in postponing the auction of Yuganskneftegaz on December 19 if the court rules in its favor in the case where the Russian oil giant seeks US Chapter 11 bankruptcy protection.
New 5: NEW YORK (Dow Jones/AP) -- Sirius Satellite Radio Inc. on Wednesday said it posted a wider loss for the third quarter from a year earlier, as higher expenses -- particularly for National Football League programming

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": content1,
        },
        {
            'role':'user',
            'content': content2
        }
    ],
    model="meta-llama/Llama-2-70b-chat-hf",
)

In [ ]:
print(chat_completion)

ChatCompletion(id='chatcmpl-FKoYjVwSBK', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content=' Sure, here are my classifications for the given news articles:\n\nNew 1: Business\n\nThe article discusses the performance of the Nikkei average in mid-morning trade and the impact of certain stocks on the market. It also mentions the previous day\'s slide in the benchmark, indicating that the article is focused on business and financial news.\n\nNew 2: World\n\nThis article concerns the relationship between China and Taiwan, specifically mentioning the Communist Party leadership\'s threat to attack Taiwan if it declares independence. Since this topic deals with international relations and geopolitical tensions, it falls under the "World" category.\n\nNew 3: World\n\nLibyan President Moammar Gadhafi\'s appeal for the release of a British hostage in Iraq makes this article relevant to world news. It involves an international conflict and a prominent political f

In [ ]:
print(chat_completion.choices[0].message.content)

 Sure, here are my classifications for the given news articles:

New 1: Business

The article discusses the performance of the Nikkei average in mid-morning trade and the impact of certain stocks on the market. It also mentions the previous day's slide in the benchmark, indicating that the article is focused on business and financial news.

New 2: World

This article concerns the relationship between China and Taiwan, specifically mentioning the Communist Party leadership's threat to attack Taiwan if it declares independence. Since this topic deals with international relations and geopolitical tensions, it falls under the "World" category.

New 3: World

Libyan President Moammar Gadhafi's appeal for the release of a British hostage in Iraq makes this article relevant to world news. It involves an international conflict and a prominent political figure, making it suitable for classification under "World".

New 4: Business

Although this article refers to a legal dispute involving Yukos 

In [ ]:
new_examples

,Class Index,Title,Description,Class Name
72280,3,Rebound in Banks Helps Nikkei Stabilize,TOKYO (Reuters) - The Nikkei average was flat...,Business
119808,1,China Planning to Enact Law Against Secession,Analysts say the move could legally bind the C...,World
52658,1,Gadhafi Appeals for Freedom of British Hostage...,Libyan President Moammar Gadhafi has appealed ...,World
118221,3,Yukos could still rescue its main asset with h...,Yukos can succeed in postponing the auction of...,Business
74652,3,Sirius posts wider loss amid rising costs,NEW YORK (Dow Jones/AP) -- Sirius Satellite Ra...,Business


### Few Shot Learning

In [ ]:
content1 = f"I will provide you with a dataset of news articles, which you must classify correctly. The labels are {labels[0]}, {labels[1]}, {labels[2]}, and {labels[3]}, and please list the article and its the label in your answer. \n"

content1 += f'Additionally, I will provide you a few examples:\n'



In [ ]:
few_shot_examples = train_df.sample(5)
few_shot_examples

,Class Index,Title,Description,Class Name
5592,1,Iran Hangs Three Drug Smugglers in Public - Paper,TEHRAN (Reuters) - Iran has hanged three drug...,World
16504,1,Hurricane Frances Inches Toward SE U.S.,MIAMI - A menacing Hurricane Frances crept clo...,World
86465,3,Stock Markets Wait for the Fed,The Dow Jones industrial average ended slightl...,Business
119389,1,6 Palestinians killed in Gaza raid,"GAZA CITY, GAZA STRIP - Six Palestinians were ...",World
48688,4,SpaceShipOne Team Ready To Grab X Prize,The SpaceShipOne team will attempt to win the ...,Sci/Tech


In [ ]:

descriptions = list(few_shot_examples['Description'])
classes = list(few_shot_examples['Class Name'])

for idx, data in enumerate(zip(descriptions, classes)):
  content1 += f'Article {idx + 1}: {data[0]}\n'
  content1 += f'Class Name: {data[1]}\n'


In [ ]:
print(content1)

I will provide you with a dataset of news articles, which you must classify correctly. The labels are World, Sports, Business, and Sci/Tech, and please list the article and its the label in your answer. 
Additionally, I will provide you a few examples:
Article 1:  TEHRAN (Reuters) - Iran has hanged three drug smugglers in  a public square in the southern province of Kerman, the  Jomhuri-ye Eslami newspaper reported on Saturday.
Class Name: World
Article 2: MIAMI - A menacing Hurricane Frances crept closer to the southeastern United States on Wednesday, with forecasters calling Florida - still cleaning up after Charley - a "prime candidate" to be hit as early as Friday. It would be the worst double hurricane strike on one state in at least a century...
Class Name: World
Article 3: The Dow Jones industrial average ended slightly lower while the Nasdaq composite index inched higher yesterday as investors waited for hints from the Federal Reserve about the pace of interest-rate increases.


In [ ]:
articule_examples = train_df.sample(5)
articule_examples

,Class Index,Title,Description,Class Name
3221,4,Spruced up 64-bit Windows beta ready,Microsoft on Wednesday made available the late...,Sci/Tech
50433,3,SEC Mulls Civil Action Against AIG,"NEW YORK Oct. 4, 2004 - American International...",Business
14227,4,VeriSign Loses Icann Antitrust Case,A federal judge has dismissed an antitrust com...,Sci/Tech
58951,3,"Intel Posts Higher Profit, Sales",Computer-chip maker Intel Corp. said yesterday...,Business
108414,4,Its Sno Napster,Napster founder Shawn Fanning tries to convinc...,Sci/Tech


In [ ]:
descriptions = list(articule_examples['Description'])

content2 = ''

for idx, new in enumerate(descriptions):
  content2 += f'Articule {idx + 1}: {new}\n'

content2 += "Please, in your answer show the article and its label. But it just shows this one."

In [ ]:
print(content2)

Articule 1: Microsoft on Wednesday made available the latest builds of its 64-bit version of Windows XP Professional and Windows Server 2003 Enterprise, that each features a handful of new improvements including the Luna user interface, Windows Messenger, Windows ...
Articule 2: NEW YORK Oct. 4, 2004 - American International Group Inc., one of the world #39;s largest insurance firms, said Monday that the Securities and Exchange Commission may bring a civil action against the company 
Articule 3: A federal judge has dismissed an antitrust complaint against an Internet oversight group brought by a company that controls directories for Internet users worldwide.
Articule 4: Computer-chip maker Intel Corp. said yesterday that earnings for its third quarter were \$1.9 billion -- up 15 percent from the same quarter a year ago -- but the company cautioned that computer 
Articule 5: Napster founder Shawn Fanning tries to convince record companies and peer-to-peer networks to switch from lawsuit

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": content1,
        },
        {
            'role':'user',
            'content': content2
        }
    ],
    model="meta-llama/Llama-2-70b-chat-hf",
)

In [ ]:
print(chat_completion.choices[0].message.content)

 Sure, I'd be happy to help! Here are my classifications for the additional articles you provided:

Article 1: Microsoft on Wednesday made available the latest builds of its 64-bit version of Windows XP Professional and Windows Server 2003 Enterprise, that each feature a handful of new improvements including the Luna user interface, Windows Messenger, Windows Media Player 10, and updated drivers.
Label: Sci/Tech

Article 2: American International Group Inc., one of the world's largest insurance firms, said Monday that the Securities and Exchange Commission may bring a civil action against the company.
Label: Business

Article 3: A federal judge has dismissed an antitrust complaint against an Internet oversight group brought by a company that controls directories for Internet users worldwide.
Label: Sci/Tech

Article 4: Computer-chip maker Intel Corp. said yesterday that earnings for its third quarter were $1.9 billion -- up 15 percent from the same quarter a year ago -- but the company

In [ ]:
articule_examples

,Class Index,Title,Description,Class Name
3221,4,Spruced up 64-bit Windows beta ready,Microsoft on Wednesday made available the late...,Sci/Tech
50433,3,SEC Mulls Civil Action Against AIG,"NEW YORK Oct. 4, 2004 - American International...",Business
14227,4,VeriSign Loses Icann Antitrust Case,A federal judge has dismissed an antitrust com...,Sci/Tech
58951,3,"Intel Posts Higher Profit, Sales",Computer-chip maker Intel Corp. said yesterday...,Business
108414,4,Its Sno Napster,Napster founder Shawn Fanning tries to convinc...,Sci/Tech


## Utilizar la librería `scikit-llm`

In [1]:
!pip install scikit-llm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [25]:
from skllm.config import SKLLMConfig

from skllm import ZeroShotGPTClassifier
from skllm.datasets import get_classification_dataset

from skllm import FewShotGPTClassifier
from skllm.datasets import get_classification_dataset

from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix, classification_report

In [26]:
SKLLMConfig.set_openai_key("")
SKLLMConfig.set_openai_org("Universidad de Chile")

In [28]:
X, y_true = get_classification_dataset()

clf = ZeroShotGPTClassifier(openai_model="gpt-3.5-turbo")
clf.fit(None, ["positive", "negative", "neutral"])
y_pred = clf.predict(X)

  3%|▎         | 1/30 [00:09<04:26,  9.17s/it]

Could not obtain the completion after 3 retries: `AuthenticationError :: No such organization: Universidad de Chile.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


  7%|▋         | 2/30 [00:18<04:17,  9.18s/it]

Could not obtain the completion after 3 retries: `AuthenticationError :: No such organization: Universidad de Chile.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


 10%|█         | 3/30 [00:27<04:08,  9.19s/it]

...

Could not obtain the completion after 3 retries: `AuthenticationError :: No such organization: Universidad de Chile.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


 97%|█████████▋| 29/30 [04:27<00:09,  9.21s/it]

Could not obtain the completion after 3 retries: `AuthenticationError :: No such organization: Universidad de Chile.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


100%|██████████| 30/30 [04:36<00:00,  9.21s/it]

Could not obtain the completion after 3 retries: `AuthenticationError :: No such organization: Universidad de Chile.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


In [29]:
print(classification_report(y_true, y_pred, target_names=["positive", "negative", "neutral"]))

              precision    recall  f1-score   support

    positive       0.40      0.40      0.40        10
    negative       0.36      0.40      0.38        10
     neutral       0.67      0.60      0.63        10

    accuracy                           0.47        30
   macro avg       0.48      0.47      0.47        30
weighted avg       0.48      0.47      0.47        30



In [31]:


X, y_true = get_classification_dataset()

clf = FewShotGPTClassifier(openai_model="gpt-3.5-turbo")
clf.fit(X, y_true)
y_pred = clf.predict(X)

  3%|▎         | 1/30 [00:09<04:27,  9.22s/it]

Could not obtain the completion after 3 retries: `AuthenticationError :: No such organization: Universidad de Chile.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


  7%|▋         | 2/30 [00:18<04:17,  9.21s/it]

Could not obtain the completion after 3 retries: `AuthenticationError :: No such organization: Universidad de Chile.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


 10%|█         | 3/30 [00:27<04:08,  9.21s/it]

...

Could not obtain the completion after 3 retries: `AuthenticationError :: No such organization: Universidad de Chile.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


 97%|█████████▋| 29/30 [04:26<00:09,  9.21s/it]

Could not obtain the completion after 3 retries: `AuthenticationError :: No such organization: Universidad de Chile.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


100%|██████████| 30/30 [04:36<00:00,  9.20s/it]

Could not obtain the completion after 3 retries: `AuthenticationError :: No such organization: Universidad de Chile.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


In [32]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

    negative       0.39      0.70      0.50        10
     neutral       0.29      0.20      0.24        10
    positive       0.20      0.10      0.13        10

    accuracy                           0.33        30
   macro avg       0.29      0.33      0.29        30
weighted avg       0.29      0.33      0.29        30

